In [68]:
# Import necessary modules
import numpy as np
import pandas as pd
from math import exp

In [69]:
# Initialize Network
def initialize_network(inputs, n_hiddens, n_hidden_neurons, n_outputs):
    # Create network with 2 hidden layers
    network = {}
    network['input_layer'] = {}
        
    n = 1
    while(n <= n_hiddens):
        network['hidden_' + str(n) + '_layer'] = {}
        if n == 1:
            network['hidden_' + str(n) + '_layer']['weights']\
                                                    = np.random.random((n_hidden_neurons, len(inputs)+1))
        else:
            network['hidden_' + str(n) + '_layer']['weights']\
                                                = np.random.random((n_hidden_neurons, n_hidden_neurons+1))
        n += 1
    
    network['output_layer']= {}    
    
    return network

In [80]:
# Initialize Network
def initialize_network(inputs, n_hiddens, n_hidden_neurons, n_outputs):
    # Create network with 2 hidden layers
    network = {}
    network['input_layer'] = []
        
    n = 1
    while(n <= n_hiddens):
        if n == 1:
            network['hidden_' + str(n) + '_layer'] = \
            [{'weights' : np.random.random((n_hidden_neurons, len(inputs)+1))}]
        
        else:
            network['hidden_' + str(n) + '_layer'] = [{'weights' : np.random.random((n_hidden_neurons, n_hidden_neurons+1))}]
        n += 1
    
    network['output_layer']= []
    
    return network

In [74]:
# Calculate neuron
def calculate_neuron(weights, inputs):
    inputs_plus_bias = np.append(inputs, 1)
    output = np.dot(weights, inputs_plus_bias)
    
    return output

In [75]:
# Activate neuron using Sigmoid 
def activate_sigmoid(output):
    return 1.0 / (1.0 + output)

In [ ]:
# Forward propagate inputs
def forward_propagate(network, inputs):
    for layer in network.keys():
        if layer == "input_layer":
            continue
        for neuron in 
                   

In [81]:
np.random.seed(2023)
inputs = np.array([14.88, 14.57, 0.8811, 5.554])
network = initialize_network(inputs, 3, 2, 3)

for key, value in network.items():
    print(key, value)

input_layer []
hidden_1_layer [{'weights': array([[0.3219883 , 0.89042245, 0.58805226, 0.12659609, 0.14134122],
       [0.46789559, 0.02208966, 0.72727471, 0.52438734, 0.54493524]])}]
hidden_2_layer [{'weights': array([[0.45637326, 0.50138226, 0.39446855],
       [0.1511723 , 0.36087518, 0.16207701]])}]
hidden_3_layer [{'weights': array([[0.33795869, 0.18032328, 0.3909914 ],
       [0.03564821, 0.56486165, 0.20346149]])}]
output_layer []


In [73]:
a = np.append(inputs, 1)
b = np.array([[0.3219883 , 0.89042245, 0.58805226, 0.12659609, 0.14134122],
              [0.46789559, 0.02208966, 0.72727471, 0.52438734, 0.54493524]])
np.dot(b, a)

array([19.12722975, 11.382317  ])